#Text Summarization
The idea of document summarization is a
bit different from keyphrase extraction or topic modeling. In this case, the end result
is still in the form of some document, but with a few sentences based on the length we
might want the summary to be. This is similar to an abstract or an executive summary
in a research paper. The main objective of automated document summarization is
to perform this summarization without involving human input, except for running
computer programs. Mathematical and statistical models help in building and
automating the task of summarizing documents by observing their content and context.

There are two broad approaches to document summarization using automated
techniques. They are described as follows:
- __Extraction-based techniques:__ These methods use mathematical
and statistical concepts like SVD to extract some key subset of the
content from the original document such that this subset of content
contains the core information and acts as the focal point of the entire
document. This content can be words, phrases, or even sentences.
The end result from this approach is a short executive summary of a
couple of lines extracted from the original document. No new content
is generated in this technique, hence the name extraction-based.
- __Abstraction-based techniques:__ These methods are more complex
and sophisticated. They leverage language semantics to create
representations and use natural language generation (NLG)
techniques where the machine uses knowledge bases and semantic
representations to generate text on its own and create summaries
just like a human would write them. Thanks to deep learning, we can
implement these techniques easily but they require a lot of data and
compute.

We will cover extraction based methods here due to constraints of needed a lot of data + compute for abstraction based methods. But you can leverage the seq2seq models you learnt in language translation on an appropriate dataset to build deep learning based abstractive summarizers

## Install necessary dependencies


#Extraction based techniques


In [1]:
import nltk
import numpy as np
import pandas as pd
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Get Text Document

We use the description of a very popular role-playing game (RPG) Skyrim from
Bethesda Softworks for summarization. 

In [2]:
DOCUMENT = """
The Elder Scrolls V: Skyrim is an action role-playing video game developed by Bethesda Game Studios 
and published by Bethesda Softworks. It is the fifth main installment in The Elder Scrolls series, 
following The Elder Scrolls IV: Oblivion.
The game's main story revolves around the player character's quest to defeat Alduin the World-Eater, 
a dragon who is prophesied to destroy the world. The game is set 200 years after the events of Oblivion 
and takes place in the fictional province of Skyrim. Over the course of the game, the player completes 
quests and develops the character by improving skills. The game continues the open-world tradition of 
its predecessors by allowing the player to travel anywhere in the game world at any time, and to ignore 
or postpone the main storyline indefinitely.
The team opted for a unique and more diverse open world than Oblivion's Imperial Province of Cyrodiil, 
which game director and executive producer Todd Howard considered less interesting by comparison. 
The game was released to critical acclaim, with reviewers particularly mentioning the character advancement 
and setting, and is considered to be one of the greatest video games of all time.


The Elder Scrolls V: Skyrim is an action role-playing game, playable from either a first or 
third-person perspective. The player may freely roam over the land of Skyrim which is an open world 
environment consisting of wilderness expanses, dungeons, cities, towns, fortresses, and villages. 
Players may navigate the game world more quickly by riding horses or by utilizing a fast-travel system 
which allows them to warp to previously discovered locations. The game's main quest can be completed or 
ignored at the player's preference after the first stage of the quest is finished. However, some quests 
rely on the main storyline being at least partially completed. Non-player characters (NPCs) populate the 
world and can be interacted with in a number of ways: the player may engage them in conversation, 
marry an eligible NPC, kill them or engage in a nonlethal "brawl". The player may 
choose to join factions which are organized groups of NPCs — for example, the Dark Brotherhood, a band 
of assassins. Each of the factions has an associated quest path to progress through. Each city and town 
in the game world has jobs that the player can engage in, such as farming.

Players have the option to develop their character. At the beginning of the game, players create 
their character by selecting their sex and choosing between one of several races including humans, 
orcs, elves, and anthropomorphic cat or lizard-like creatures and then customizing their character's 
appearance. Over the course of the game, players improve their character's skills which are numerical 
representations of their ability in certain areas. There are eighteen skills divided evenly among the 
three schools of combat, magic, and stealth. When players have trained skills enough to meet the 
required experience, their character levels up. Health is depleted primarily when the player 
takes damage and the loss of all health results in death. Magicka is depleted by the use of spells, 
certain poisons and by being struck by lightning-based attacks. Stamina determines the player's 
effectiveness in combat and is depleted by sprinting, performing heavy "power attacks" 
and being struck by frost-based attacks. Skyrim is the first entry in The Elder Scrolls to 
include dragons in the game's wilderness. Like other creatures, dragons are generated randomly in 
the world and will engage in combat with NPCs, creatures and the player. Some dragons may attack 
cities and towns when in their proximity. The player character can absorb the souls of dragons 
in order to use powerful spells called "dragon shouts" or "Thu'um". A regeneration 
period limits the player's use of shouts in gameplay.

Skyrim is set around 200 years after the events of The Elder Scrolls IV: Oblivion, although it is 
not a direct sequel. The game takes place in Skyrim, a province of the Empire on the continent of 
Tamriel, amid a civil war between two factions: the Stormcloaks, led by Ulfric Stormcloak, and the 
Imperial Legion, led by General Tullius. The player character is a Dragonborn, a mortal born with 
the soul and power of a dragon. Alduin, a large black dragon who returns to the land after being 
lost in time, serves as the game's primary antagonist. Alduin is the first dragon created by Akatosh, 
one of the series' gods, and is prophesied to destroy and consume the world.
"""

In [3]:
import re
DOCUMENT = re.sub(r'\n|\r', ' ', DOCUMENT) #Combining all the paragraphs
DOCUMENT = re.sub(r' +', ' ', DOCUMENT)
DOCUMENT = DOCUMENT.strip()

In [4]:
print(DOCUMENT)

The Elder Scrolls V: Skyrim is an action role-playing video game developed by Bethesda Game Studios and published by Bethesda Softworks. It is the fifth main installment in The Elder Scrolls series, following The Elder Scrolls IV: Oblivion. The game's main story revolves around the player character's quest to defeat Alduin the World-Eater, a dragon who is prophesied to destroy the world. The game is set 200 years after the events of Oblivion and takes place in the fictional province of Skyrim. Over the course of the game, the player completes quests and develops the character by improving skills. The game continues the open-world tradition of its predecessors by allowing the player to travel anywhere in the game world at any time, and to ignore or postpone the main storyline indefinitely. The team opted for a unique and more diverse open world than Oblivion's Imperial Province of Cyrodiil, which game director and executive producer Todd Howard considered less interesting by comparison.

Sentences Collection

In [5]:
sentences = nltk.sent_tokenize(DOCUMENT)
print(sentences)
print("No. of sentences:",len(sentences))

['The Elder Scrolls V: Skyrim is an action role-playing video game developed by Bethesda Game Studios and published by Bethesda Softworks.', 'It is the fifth main installment in The Elder Scrolls series, following The Elder Scrolls IV: Oblivion.', "The game's main story revolves around the player character's quest to defeat Alduin the World-Eater, a dragon who is prophesied to destroy the world.", 'The game is set 200 years after the events of Oblivion and takes place in the fictional province of Skyrim.', 'Over the course of the game, the player completes quests and develops the character by improving skills.', 'The game continues the open-world tradition of its predecessors by allowing the player to travel anywhere in the game world at any time, and to ignore or postpone the main storyline indefinitely.', "The team opted for a unique and more diverse open world than Oblivion's Imperial Province of Cyrodiil, which game director and executive producer Todd Howard considered less intere

## Basic Text pre-processing

In [6]:
stop_words = nltk.corpus.stopwords.words('english')

# Handeling 'not' words from stop_words
def handleStops(stop_words):
  rm = ['not', 'no', 'nor', 'against']

  for i in stop_words:
    if "n'" in i:
      rm.append(i)

  for i in rm:
    stop_words.remove(i)
  return stop_words

stop_words = handleStops(stop_words)

In [7]:
def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document) #Function Def Vectorize
norm_sentences = normalize_corpus(sentences)
norm_sentences[:3]

array(['elder scrolls v skyrim action roleplaying video game developed bethesda game studios published bethesda softworks',
       'fifth main installment elder scrolls series following elder scrolls iv oblivion',
       'games main story revolves around player characters quest defeat alduin worldeater dragon prophesied destroy world'],
      dtype='<U183')

## *1. UNI-GRAM*

In [8]:
# Creation of Unigarms
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range =(1, 1))
X1 = vectorizer.fit_transform(norm_sentences)
unigrams = (vectorizer.get_feature_names())
print(unigrams)
print(len(unigrams))

['ability', 'absorb', 'acclaim', 'action', 'advancement', 'akatosh', 'alduin', 'allowing', 'allows', 'although', 'amid', 'among', 'antagonist', 'anthropomorphic', 'anywhere', 'appearance', 'areas', 'around', 'assassins', 'associated', 'attack', 'attacks', 'band', 'beginning', 'bethesda', 'black', 'born', 'brawl', 'brotherhood', 'called', 'cat', 'certain', 'character', 'characters', 'choose', 'choosing', 'cities', 'city', 'civil', 'combat', 'comparison', 'completed', 'completes', 'considered', 'consisting', 'consume', 'continent', 'continues', 'conversation', 'course', 'create', 'created', 'creatures', 'critical', 'customizing', 'cyrodiil', 'damage', 'dark', 'death', 'defeat', 'depleted', 'destroy', 'determines', 'develop', 'developed', 'develops', 'direct', 'director', 'discovered', 'diverse', 'divided', 'dragon', 'dragonborn', 'dragons', 'dungeons', 'effectiveness', 'eighteen', 'either', 'elder', 'eligible', 'elves', 'empire', 'engage', 'enough', 'entry', 'environment', 'evenly', 'eve

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


###TF

In [9]:
print("Norm Sentences:", norm_sentences)
print("Len of Norm Sentences:", len(norm_sentences))

Norm Sentences: ['elder scrolls v skyrim action roleplaying video game developed bethesda game studios published bethesda softworks'
 'fifth main installment elder scrolls series following elder scrolls iv oblivion'
 'games main story revolves around player characters quest defeat alduin worldeater dragon prophesied destroy world'
 'game set years events oblivion takes place fictional province skyrim'
 'course game player completes quests develops character improving skills'
 'game continues openworld tradition predecessors allowing player travel anywhere game world time ignore postpone main storyline indefinitely'
 'team opted unique diverse open world oblivions imperial province cyrodiil game director executive producer todd howard considered less interesting comparison'
 'game released critical acclaim reviewers particularly mentioning character advancement setting considered one greatest video games time'
 'elder scrolls v skyrim action roleplaying game playable either first thirdp

In [10]:
def computeTF(doc):
  valTF = []
  for each in doc:
    wordDict = dict.fromkeys(unigrams, 0)
    sentence = each.split(" ")

    for word in sentence:
      if len(word) == 1:
        sentence.remove(word)

    for word in sentence:
      wordDict[word]+=1

    res = []
    for i in wordDict:
      comp = float(wordDict[i] / len(each))
      res.append(round(comp, 4))
    
    valTF.append(res)
  return(valTF)
    
TF = computeTF(norm_sentences)

In [11]:
TF = np.array(TF)
TF = TF.T
type(TF)

numpy.ndarray

In [12]:
df = pd.DataFrame(TF, index=unigrams)
df.sort_index(ascending=True).head(10)

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
ability,0.0000,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0000,0.0000,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.000,0.0,0.0,0.0000,0.0000
absorb,0.0000,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0000,0.0000,0.0,...,0.0,0.0,0.0,0.0111,0.0,0.000,0.0,0.0,0.0000,0.0000
acclaim,0.0000,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0074,0.0000,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.000,0.0,0.0,0.0000,0.0000
action,0.0088,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0000,0.0109,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.000,0.0,0.0,0.0000,0.0000
advancement,0.0000,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0074,0.0000,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.000,0.0,0.0,0.0000,0.0000
akatosh,0.0000,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0000,0.0000,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.000,0.0,0.0,0.0000,0.0119
alduin,0.0000,0.0,0.0088,0.0,0.0,0.0000,0.0,0.0000,0.0000,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.000,0.0,0.0,0.0125,0.0119
allowing,0.0000,0.0,0.0000,0.0,0.0,0.0072,0.0,0.0000,0.0000,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.000,0.0,0.0,0.0000,0.0000
allows,0.0000,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0000,0.0000,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.000,0.0,0.0,0.0000,0.0000
although,0.0000,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0000,0.0000,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.012,0.0,0.0,0.0000,0.0000


###DF (Document Frequency/Term Presence)

In [13]:
DF = vectorizer.fit_transform(norm_sentences)
DF = DF.toarray()
DF = DF.T
DF

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [14]:
df = pd.DataFrame(DF, index=unigrams)
df

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
ability,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
absorb,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
acclaim,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
action,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
advancement,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ways,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
wilderness,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
world,0,0,1,0,0,1,1,0,0,1,...,0,1,0,0,0,0,0,0,0,1
worldeater,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


###TF-IDF

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
dt_matrix = tv.fit_transform(norm_sentences)
dt_matrix = dt_matrix.toarray()
dt_matrix

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.18074668, 0.30828241,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.19328565, 0.        ,
        0.        ]])

In [16]:
td_matrix = dt_matrix.T #Transpose Matrix
td_matrix
print(td_matrix.shape)

(271, 35)


In [17]:
pd.DataFrame(np.round(td_matrix, 2), index=unigrams)

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
ability,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00
absorb,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.0,0.31,0.0,0.00,0.0,0.0,0.0,0.00
acclaim,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.28,0.00,0.00,...,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00
action,0.25,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.32,0.00,...,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00
advancement,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.28,0.00,0.00,...,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ways,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00
wilderness,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.24,...,0.37,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00
world,0.00,0.0,0.18,0.00,0.0,0.16,0.14,0.00,0.00,0.16,...,0.00,0.19,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.19
worldeater,0.00,0.0,0.31,0.00,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00


## *2. BI-GRAM* 

In [18]:
# Forming Bigrams
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range = (2,2))
bigrams_matrix = vectorizer.fit_transform(norm_sentences) 
bigrams = (vectorizer.get_feature_names())
print(bigrams)

['ability certain', 'absorb souls', 'acclaim reviewers', 'action roleplaying', 'advancement setting', 'akatosh one', 'alduin first', 'alduin large', 'alduin worldeater', 'allowing player', 'allows warp', 'although not', 'amid civil', 'among three', 'anthropomorphic cat', 'anywhere game', 'around player', 'around years', 'associated quest', 'attack cities', 'attacks struck', 'band assassins', 'beginning game', 'bethesda game', 'bethesda softworks', 'black dragon', 'born soul', 'brotherhood band', 'called dragon', 'cat lizardlike', 'certain areas', 'certain poisons', 'character absorb', 'character advancement', 'character dragonborn', 'character improving', 'character levels', 'character selecting', 'characters appearance', 'characters npcs', 'characters quest', 'characters skills', 'choose join', 'choosing one', 'cities towns', 'city town', 'civil war', 'combat depleted', 'combat magic', 'combat npcs', 'completed ignored', 'completes quests', 'considered less', 'considered one', 'consis

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### TF-IDF

In [19]:
# Applying TFIDF
vectorizer = TfidfVectorizer(ngram_range = (2, 2))
bigram_matrix = vectorizer.fit_transform(norm_sentences)
bigram_matrix = bigram_matrix.toarray()
bigram_matrix = bigram_matrix.T
print("\n\nScores : \n", bigram_matrix)



Scores : 
 [[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.27111489 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [20]:
pd.DataFrame(np.round(bigram_matrix, 2), index=bigrams)

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
ability certain,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
absorb souls,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,...,0.0,0.0,0.0,0.29,0.0,0.00,0.0,0.0,0.0,0.0
acclaim reviewers,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.26,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
action roleplaying,0.26,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.3,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
advancement setting,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.26,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
world oblivions,0.00,0.0,0.00,0.00,0.0,0.00,0.23,0.00,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
world quickly,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
world time,0.00,0.0,0.00,0.00,0.0,0.25,0.00,0.00,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
worldeater dragon,0.00,0.0,0.27,0.00,0.0,0.00,0.00,0.00,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0


## *3. TRI-GRAMS*

In [21]:
# Forming Trigrams
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range = (3,3))
trigrams_matrix = vectorizer.fit_transform(norm_sentences) 
trigrams = (vectorizer.get_feature_names())
print(trigrams)

['ability certain areas', 'absorb souls dragons', 'acclaim reviewers particularly', 'action roleplaying game', 'action roleplaying video', 'advancement setting considered', 'akatosh one series', 'alduin first dragon', 'alduin large black', 'alduin worldeater dragon', 'allowing player travel', 'allows warp previously', 'although not direct', 'amid civil war', 'among three schools', 'anthropomorphic cat lizardlike', 'anywhere game world', 'around player characters', 'around years events', 'associated quest path', 'attack cities towns', 'attacks struck frostbased', 'beginning game players', 'bethesda game studios', 'black dragon returns', 'born soul power', 'brotherhood band assassins', 'called dragon shouts', 'cat lizardlike creatures', 'certain poisons struck', 'character absorb souls', 'character advancement setting', 'character dragonborn mortal', 'character improving skills', 'character selecting sex', 'characters npcs populate', 'characters quest defeat', 'characters skills numerica

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### TF-IDF

In [22]:
# Applying TFIDF
vectorizer = TfidfVectorizer(ngram_range = (3, 3))
trigrams_matrix = vectorizer.fit_transform(norm_sentences)
trigrams_matrix = trigrams_matrix.toarray()
trigrams_matrix = trigrams_matrix.T
print("\n\nScores : \n", trigrams_matrix)



Scores : 
 [[0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 ...
 [0.        0.        0.2773501 ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]]


In [23]:
pd.DataFrame(np.round(trigrams_matrix, 2), index=trigrams)

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
ability certain areas,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
absorb souls dragons,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.3,0.0,0.00,0.0,0.0,0.0,0.0
acclaim reviewers particularly,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.27,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
action roleplaying game,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.34,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
action roleplaying video,0.3,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
world quickly riding,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
world time ignore,0.0,0.0,0.00,0.00,0.0,0.26,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
worldeater dragon prophesied,0.0,0.0,0.28,0.00,0.0,0.00,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
years events elder,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.31,0.0,0.0,0.0,0.0


## *N-Gram*

### Hot-Coded N-gram

In [24]:
def generate_ngrams(inDocs, n):
  res = []
  for doc in inDocs:
    tokens = [token for token in doc.split(" ") if token != ""]
    ngrams = zip(*[tokens[i:] for i in range(n)])
    sen =  [" ".join(ngram) for ngram in ngrams]
    res.extend(sen)
  return(res)

x = generate_ngrams(norm_sentences, 5)

In [25]:
len(x)

284

### N-Gram from module

In [26]:
# Forming ngrams
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range = (5,5))
ngrams_matrix = vectorizer.fit_transform(norm_sentences) 
ngrams = (vectorizer.get_feature_names())
print(ngrams)

['absorb souls dragons order use', 'acclaim reviewers particularly mentioning character', 'action roleplaying game playable either', 'action roleplaying video game developed', 'advancement setting considered one greatest', 'akatosh one series gods prophesied', 'alduin first dragon created akatosh', 'alduin large black dragon returns', 'alduin worldeater dragon prophesied destroy', 'allowing player travel anywhere game', 'allows warp previously discovered locations', 'amid civil war two factions', 'among three schools combat magic', 'anthropomorphic cat lizardlike creatures customizing', 'anywhere game world time ignore', 'around player characters quest defeat', 'around years events elder scrolls', 'beginning game players create character', 'bethesda game studios published bethesda', 'black dragon returns land lost', 'cat lizardlike creatures customizing characters', 'certain poisons struck lightningbased attacks', 'character absorb souls dragons order', 'character advancement setting c

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### TF-IDF N-Gram

In [27]:
# Applying TF - IDF
vectorizer = TfidfVectorizer(ngram_range = (5, 5))
ngrams_matrix = vectorizer.fit_transform(norm_sentences)
ngrams_matrix = ngrams_matrix.toarray()
ngrams_matrix = ngrams_matrix.T
print("\n\nScores : \n", trigrams_matrix)



Scores : 
 [[0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 ...
 [0.        0.        0.2773501 ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]]


In [28]:
pd.DataFrame(np.round(ngrams_matrix, 3), index=ngrams)

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
absorb souls dragons order use,0.000,0.0,0.000,0.000,0.0,0.000,0.0,0.000,0.000,0.0,...,0.0,0.0,0.0,0.333,0.0,0.000,0.0,0.0,0.0,0.0
acclaim reviewers particularly mentioning character,0.000,0.0,0.000,0.000,0.0,0.000,0.0,0.289,0.000,0.0,...,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0
action roleplaying game playable either,0.000,0.0,0.000,0.000,0.0,0.000,0.0,0.000,0.383,0.0,...,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0
action roleplaying video game developed,0.319,0.0,0.000,0.000,0.0,0.000,0.0,0.000,0.000,0.0,...,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0
advancement setting considered one greatest,0.000,0.0,0.000,0.000,0.0,0.000,0.0,0.289,0.000,0.0,...,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
world quickly riding horses utilizing,0.000,0.0,0.000,0.000,0.0,0.000,0.0,0.000,0.000,0.0,...,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0
world time ignore postpone main,0.000,0.0,0.000,0.000,0.0,0.277,0.0,0.000,0.000,0.0,...,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0
worldeater dragon prophesied destroy world,0.000,0.0,0.302,0.000,0.0,0.000,0.0,0.000,0.000,0.0,...,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0
years events elder scrolls iv,0.000,0.0,0.000,0.000,0.0,0.000,0.0,0.000,0.000,0.0,...,0.0,0.0,0.0,0.000,0.0,0.333,0.0,0.0,0.0,0.0


# Bag of Words Model

In [29]:
import re
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [36]:
df_imdb = pd.read_csv("https://raw.githubusercontent.com/kosuridivesh/NLP-TheProject/main/Dataset/imdb_labelled.txt?token=GHSAT0AAAAAABVDXGIMNHIVTEBHXJ5WBJTWYVASPEA", delimiter='\t', header=None)
df_imdb.columns = ["Review_text", "Review_class"]

df_amazon = pd.read_csv("https://raw.githubusercontent.com/kosuridivesh/NLP-TheProject/main/Dataset/amazon_cells_labelled.txt?token=GHSAT0AAAAAABVDXGIMUPDTVU3UXJT2BW42YVASOXQ", delimiter='\t', header=None)
df_amazon.columns = ["Review_text", "Review_class"]

df = pd.concat([df_imdb, df_amazon])
df

,Review_text,Review_class
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1
...,...,...
995,The screen does get smudged easily because it ...,0
996,What a piece of junk.. I lose more calls on th...,0
997,Item Does Not Match Picture.,0
998,The only thing that disappoint me is the infra...,0


In [37]:
def clean_text(df):
    df_reviews = list()
    lines = df["Review_text"].values.tolist()
    for text in lines:
        text = text.lower()
        pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        text = pattern.sub('', text)
        text = re.sub(r"[,.\"!@#$%^&*(){}?/;`~:<>+=-]", "", text)
        tokens = word_tokenize(text)
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
        PS = PorterStemmer()
        words = [PS.stem(w) for w in words if not w in stop_words]
        words = ' '.join(words)
        df_reviews.append(words)
    return df_reviews

df_reviews = clean_text(df)
df_reviews[0:20]

['slowmov aimless movi distress drift young man',
 'not sure lost flat charact audienc nearli half walk',
 'attempt arti black white clever camera angl movi disappoint becam even ridicul act poor plot line almost nonexist',
 'littl music anyth speak',
 'best scene movi gerardo tri find song keep run head',
 'rest movi lack art charm mean empti work guess empti',
 'wast two hour',
 'saw movi today thought good effort good messag kid',
 'bit predict',
 'love cast jimmi buffet scienc teacher',
 'babi owl ador',
 'movi show lot florida best made look appeal',
 'song best muppet hilari',
 'cool',
 'right case movi deliv everyth almost right face',
 'averag act main person low budget clearli see',
 'review long overdu sinc consid tale two sister singl greatest film ever made',
 'put gem against movi term screenplay cinematographi act postproduct edit direct aspect filmmak',
 'practic perfect true masterpiec sea faux masterpiec',
 'structur film easili tightli construct histori cinema think n

In [38]:
len(stop_words)

157

In [47]:
from sklearn.feature_extraction.text import CountVectorizer
CV = CountVectorizer(min_df=3)   
X = CV.fit_transform(df_reviews).toarray()
y = df["Review_class"].to_numpy()
print(np.shape(X))
print(np.shape(y))

(1748, 890)
(1748,)


In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# from sklearn.tree import DecisionTreeClassifier
# model = DecisionTreeClassifier(criterion="entropy", random_state=41)
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

from sklearn.metrics import accuracy_score, f1_score, precision_score
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
print(precision_score(y_test, y_pred))

0.6914285714285714
0.7172774869109948
0.6555023923444976
